# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [1]:
!pip install scann

     |████████████████████████████████| 10.6 MB 5.2 MB/s 


In [2]:
import numpy as np
import pandas as pd
import scann

In [3]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [4]:
config=pd.read_csv('config.csv').iloc[0]
print(config)

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object


In [5]:
user=config['id']
k_value=config['k']
print(user)
print(k_value)

276633
5


### Read the Data

In [6]:
with open(config['dataset_file'],'r') as inFile:
  appreciate_data = inFile.readlines()

len(appreciate_data)

1000000

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [7]:
user_likes=dict()
different_image=dict()
dict_to_store_index=dict()
index=0
index_key=0
key_user=-1
for line in appreciate_data:
  line = line.strip()
  user_id = int(line.split()[0])
  item_id = int(line.split()[1])
  if user_id not in user_likes:
    user_likes[user_id] = list()
    if user_id==user:
      key_user=index_key
    dict_to_store_index[index_key]=user_id
    index_key=index_key+1
  user_likes[user_id].append(item_id)
  if item_id not in different_image:
    different_image[item_id]=index
    index=index+1
  #user_likes[user_id].append(item_id)
#user_likes
#user_likes
print(len(different_image))
#len(user_likes)
print(user)
len(user_likes)


178788
276633


63497

In [8]:
diff_user=len(user_likes)
diff_image=len(different_image)
dense_matrix=np.zeros((diff_user,diff_image),dtype=np.int8)

In [9]:
row=0
for i in list(user_likes.keys()):
  for j in user_likes[i]:
    dense_matrix[row][different_image[j]]=1
  row=row+1

In [10]:
from scipy.sparse import csr_matrix
sparse_mat=csr_matrix(dense_matrix)

In [11]:
del(dense_matrix)

In [12]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)
sparse_matrix=svd.fit_transform(sparse_mat)

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [13]:
def answer(user,k_value):
  searcher = scann.scann_ops_pybind.builder(sparse_matrix, 10, "dot_product").tree(
      num_leaves=2000, num_leaves_to_search=100, training_sample_size=len(sparse_matrix)).score_ah(
      2, anisotropic_quantization_threshold=0.2).reorder(100).build()
      
  neighbors, distances = searcher.search(sparse_matrix[key_user], final_num_neighbors=k_value+1)
  ans_list=list()
  
  row=1
  temp=list()
  while row<len(neighbors):
    temp.append(neighbors[row])
    row=row+1
  for i in temp:
    ans_list.append(dict_to_store_index[i])
  return ans_list


### Answer the following questions:

#### Q1. **Explain how your choice of library works**

*I have choosen ScaNN library to implement this as it performs vector search in three phases:-*

*   Partition Phase:-In this phase firstly it splits the dataset during training time and at the query time it selects the top partitions from the partitioned dataset to pass onto the scoring stage.

*   Scoring Phase:-In this phase scann calculates the distance between the query as well as to all the data points
in the dataset(if the partition is not enabled) or to all the datapoints in the partition to search(if partition is enabled).


*   Rescoring Phase:- In this phase ScaNN takes the best k' distances from scoring and recalculates them more precisely. The top k are chosen from these k' re-computed distances. This step is highly recommended.




<br><br>

#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*I choose ScaNN library as it works fast when we compared to vanilla KNN because as ScaNN splits the dataset into different partition due to which it takes less searching spcae but in vanilla KNN searching space is large due to which time taken is more.*

*When we compare the recall value, vanilla KNN is having higher value but we can also get the same ercall value for ScaNN if we can compromise with the speed.*

<br><br>

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

The graph is comparing speed or accuracy of all the three and out of all ScaNN is faster than other two
ScaNN < faiss < annoy 
This is the order of the all the three when compared as respect to time complexity.

From [ann-benchmarks.com](http://ann-benchmarks.com/)![data.png](https://github.com/google-research/google-research/raw/master/scann/docs/glove_bench.png)

Graph indicate that for particular Accuracy out of all three ScaNN performs faster when compare with others.

For a particular value of speed Recall value of ScaNN is high as compared to others so that's why i choose ScaNN as for a particular speed it gave the better accuracy.
For a particular speed Recall value of ScaNN is highest and annoy having lowest value out of all the three.



### Open the output file to write all the lines to the file

In [14]:
outFile = open(config['output_file'], 'w')

for n_user in answer(user,k_value):
    # user_id = list(user_likes.keys())[n_user]
    main_set=set(user_likes[user])
    ultimate_set=set(user_likes[n_user])
    final_set=ultimate_set.difference(main_set)
    for item_id in final_set:
        outFile.write(str(item_id) + ' ' + str(n_user) + '\n')

outFile.close()